**The Euclidean Distance**


In [241]:
import numpy as np

# def distance(a, b):
#     return np.sqrt(np.sum(np.pow((a - b), 2)))
def distance(a, b):
    return np.sum(np.abs(a - b))

**Dynamic Programming Algorithm**


In [242]:
def dtw_matrix(grp1, grp2):
    n, m = grp1.shape[0], grp2.shape[0]
    
    # Creating the matrix
    matrix_dtw = np.zeros((m, n))
    
    #Initialization
    matrix_dtw[0, 0] = 2 * distance(grp1[0], grp2[0])
    matrix_dtw[0,1:] = float('inf')
    
    # filling the first line
    for k in range(1, m):
        matrix_dtw[k,0] = matrix_dtw[k-1,0] + distance(grp1[0],grp2[k])
    
    # Filling the matrix following the rule of DTW
    for i in range(1,m):
        for j in range(1,n):
            dist = distance(grp2[i], grp1[j])
            
            dpp1 = matrix_dtw[i-1, j] + dist
            dpp2 = matrix_dtw[i-1, j-1] + 2 * dist
            dpp3 = matrix_dtw[i, j-1] + dist
            matrix_dtw[i,j] = min(dpp1,dpp2,dpp3)

    return matrix_dtw

**Tracing(or tracking, I'm not sure about the word) The Optimum Path**

In [ ]:
def path_trace(matrix_dtw):
    chemin = 0
    return chemin

Exercice 4 : Visualisation :
* visualiser les séries et le chemin d’alignement en utilisant Matplotlib.
Cela vous aidera à voir comment les points de deux séries sont alignés.

In [245]:
import matplotlib.pyplot as plt
def visualiser_alignement(serie1, serie2, chemin):
    plt.figure(figsize=(10, 6))
    plt.plot(range(n), [0] * n, 'ro-', label='Série 1 (s1)', markersize=10)  # Série 1
    plt.plot(range(m), [1] * m, 'bo-', label='Série 2 (s2)', markersize=10)  # Série 2

# Tracer le chemin d'alignement
    for (i, j) in chemin_optimal:
        plt.plot([i, j], [0, 1], 'k--')  # Lignes pointillées entre les points

# Annotations
    plt.xticks(range(max(n, m)), [s1[i] if i < n else '' for i in range(max(n, m))])
    plt.yticks([0, 1], ['Série 1', 'Série 2'])
    plt.title("Visualisation de l'Alignement Optimal")
    plt.xlabel("Position dans la Série 1")
    plt.ylabel("Position dans la Série 2")
    plt.legend()
    plt.grid()
    plt.show()

Exercice final : Comparaison de séries :
* Voici deux séries temporelles, calculer la distance DTW, de trouver le chemin d’alignement optimal, et de visualiser les résultats.


In [246]:
serie1 = np.array([1, 2, 3, 4, 5])
serie2 = np.array([2, 3, 4, 5, 6, 7])

matrice = dtw_matrix(serie1, serie2)
print(f"La matrice: \n{matrice}")

chemin = chemin_optimal(matrice)
# print(f"Chemin: \n{chemin}")
# visualiser_alignement(serie1, serie2, chemin)

La matrice: 
[[ 2. inf inf inf inf]
 [ 4.  4.  4.  5.  7.]
 [ 7.  6.  5.  4.  5.]
 [11.  9.  7.  5.  4.]
 [16. 13. 10.  7.  5.]
 [22. 18. 14. 10.  7.]]


In [247]:
A = np.matrix([[2, 3,  4],
               [3, 3.5,4]])

T = np.matrix([[5,6],
               [5,4]])

At = A.T
Tt = T.T
mat = dtw_matrix(At, Tt)
mat

array([[10., inf, inf],
       [15., 17., 19.]])